<a href="https://colab.research.google.com/github/joonasrooben/OMML_JAB/blob/main/OMML_projet_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#OMML PROJECT

### Imports

In [339]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import math
import scipy as sp
from scipy.optimize import minimize

### Data analysis

In [196]:
data = pd.read_csv('DATA.csv')

In [197]:
data.head(5)

,x1,x2,y
0,1.725110,2.665940,0.101883
1,-1.137973,-1.059821,2.172760
2,1.652338,2.958089,0.118859
3,1.866858,-1.740966,-0.289316
4,-1.760604,1.350072,0.104652


In [198]:
len(data)

250

### Split the dataset

We put 0.255 for the test_size instead of 0.25 to be sure to have 186 observations in the training_data

In [199]:
X_train, X_test, y_train, y_test = train_test_split(data[['x1','x2']], data[['y']], test_size=0.255, random_state=1990243)

In [200]:
n = len(X_train)
n

186

In [201]:
X_train

,x1,x2
236,0.890055,-2.955518
134,0.355821,-2.385734
129,1.795969,0.914778
209,0.070684,-2.177204
52,1.674262,0.182846
...,...,...
244,1.557484,-0.167087
158,-1.730929,-0.657224
33,-1.367034,2.838806
35,-1.733677,-1.672311


### Question 1


#### Question 1.1


**Activation function**

We define the acivation function.

In [220]:
def activationFunction(t, sigma = 1):
  return (np.exp(2*sigma*t)-1)/(np.exp(2*sigma*t)+1)

**Weight Initialization for Tanh**

The current standard approach for initialization of the weights of neural network layers and nodes that use the Sigmoid or TanH activation function is called “glorot” or “xavier” initialization

The xavier initialization method is calculated as a random number with a uniform probability distribution (U) between the range -(1/sqrt(n)) and 1/sqrt(n), where n is the number of inputs to the node.

weight = U [-(1/sqrt(n)), 1/sqrt(n)]

Let's define all the hyperparameters here
Let's put 3 neurons in our hidden layer

In [308]:
N = 5
rho = 1
sigma = 1

In [309]:
inputs = X_train
inputs = inputs.to_numpy()
inputs.shape

(186, 2)

In [310]:
def createWeightsForOneNeuron(N):
  initial_weights_one_neuron = []
  for i in range(0,N):
    initial_weights_one_neuron.append(np.random.uniform(low = -(1/math.sqrt(N)), high = 1/math.sqrt(N)))
  return initial_weights_one_neuron

def create_initial_weights(N):
  initial_weights = []
  for i in range(0,2):
    initial_weights.append(createWeightsForOneNeuron(N))
  return initial_weights

weightsW = np.asarray(create_initial_weights(N))


In [311]:
mult = np.matmul(inputs, weightsW)

Create vector of bias


In [312]:
bias = np.ones((N,1))

Creation of the linear function

∑wjixi+bj


In [313]:
linearResult = mult + bias.T
linearResult.shape

(186, 5)

Now the step is the get this number into the activation function

In [314]:
acti = activationFunction(linearResult)
acti.shape

(186, 5)

We generate weigths for hidden layers


In [315]:
def createWeightsForHiddenLayer(N):
  weightHiddenLayer = []
  for i in range(0,N):
    weightHiddenLayer.append(np.random.uniform(low = -(1/math.sqrt(N)), high = 1/math.sqrt(N)))
  return weightHiddenLayer

In [316]:
v = createWeightsForHiddenLayer(N)
v = np.array(v)

In [317]:
f_x = np.matmul(v,acti.T)
f_x.shape


(186,)

Now we have the MLP. We'll computed the regularized training error function E(v,w,b)

In [318]:
y = y_train
y = y.to_numpy()
y = y.reshape((-1))
y.shape

(186,)

In [319]:
P = len(X_train)

In [320]:
firstTermE = (1/2*P)* sum((f_x - y)**2)

First create the regularization term



In [321]:
omega = [v, weightsW, bias]

In [322]:

def computeRegularizationTerm(omega):
    normV = np.linalg.norm(omega[0]**2)
    normW = np.linalg.norm(omega[1]**2)
    normB = np.linalg.norm(omega[2]**2)
    return normV + normW + normB

omega_norm = computeRegularizationTerm(omega)

regularizationTerm = (rho/2)*(omega_norm)

In [323]:
E = firstTermE + regularizationTerm
E

27621.751689776534

To resume a big function we have 

In [357]:
def expectationFunction(omega ,input, target, N, rho, sigma):

  W = omega[0]
  v = omega[1]
  bias = omega[2]

  x = input.to_numpy()

  mult = np.matmul(inputs, W)

  linearResult = mult + bias.T

  acti = activationFunction(linearResult, sigma)

  f_x = np.matmul(v,acti.T)

  y = target.to_numpy()
  y = y.reshape((-1)) 

  P = len(input)

  firstTermE = (1/2*P)* sum((f_x - y)**2)

  omega = [v, W, bias]

  regularizationTerm = (rho/2)*(omega_norm)

  E = firstTermE + regularizationTerm

  return  E

In [358]:
N = 4
rho = 1e-5
sigma = 1
W = np.asarray(create_initial_weights(N))
v = createWeightsForHiddenLayer(N)
bias = np.ones((N,1))
omega = [W ,v ,bias]

In [360]:
expectationFunction(omega, X_train ,y_train ,N ,rho ,sigma)

44254.861127044074

In [369]:
res = minimize(expectationFunction, omega, args=((X_train ,y_train ,N ,rho ,sigma)), method = 'nelder-mead')

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


ValueError: ignored